# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
pip install --upgrade azureml-core

     |████████████████████████████████| 3.3 MB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 45.1 MB/s eta 0:00:01
ERROR: ray 2.0.0 has requirement click<=8.0.4,>=7.0, but you'll have click 8.1.3 which is incompatible.
ERROR: ray 2.0.0 has requirement grpcio<=1.43.0,>=1.28.1; python_version < "3.10", but you'll have grpcio 1.54.2 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jinja2>=3.0.3, but you'll have jinja2 2.11.2 which is incompatible.
ERROR: google-auth 2.18.1 has requirement urllib3<2.0, but you'll have urllib3 2.2.1 which is incompatible.
ERROR: datasets 2.3.2 has requirement dill<0.3.6, but you'll have dill 0.3.6 which is incompatible.
ERROR: databricks-cli 0.17.7 has requirement urllib3<2.0.0,>=1.26.7, but you'll have urllib3 2.2.1 which is incompatible.
ERROR: dask-sql 2023.6.0 has requirement pandas>=1.4.0, but you'll have pandas 1.1.5 which is incompatible.
ERROR: botocore 1.23.19 has requirement urllib3<1.27,>=1.25.4, but yo

In [2]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, loguniform
from azureml.core import Environment, ScriptRunConfig
import os
import pandas as pd
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
import logging
import csv
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.pipeline.steps import AutoMLStep
import joblib
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core import Model

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.get(name = 'quick-starts-ws-260697',
                   subscription_id = '81cefad3-d2c9-4f77-a466-99a7f541c7bb',
                   resource_group = 'aml-quickstarts-260697')

# choose a name for experiment
experiment_name = 'automl-heart'

experiment=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-260697
Azure region: westus2
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-260697


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
cluster_name = "notebook260697"
project_folder = './capstone'

found = False
description_text = "Heart Data for Udacity Capstone"

dataset = Dataset.get_by_name(ws, name='heart-data')

try:
    aml_compute = ComputeTarget(workspace=ws, name = cluster_name)
    print("Found existing cluster, use it")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2", max_nodes= 4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=cluster_name,
                             task = "classification",
                             training_data=dataset,
                             #time_column_name="date", 
                             label_column_name="DEATH_EVENT",  
                             #path = project_folder,
                             #enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

Found existing cluster, use it


In [4]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl-heart,AutoML_33f1d47c-9daf-4661-bad0-9711d7677fac,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
# remote_run = experiment.submit(automl_config, show_output = False)

# RunDetails(remote_run).show()

# automl_run = experiment.start_logging()

# automl_run.log(name="message", value = "Hello from run!")

# print(automl_run.get_status())

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [10]:
best_run_metrics_automl = remote_run.get_metrics()
print(best_run_metrics_automl["accuracy"])

0.8863276836158193


In [11]:
#TODO: Save the best model
model = remote_run.register_model(model_name = "automl-heart-sdk", metric="accuracy")

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
from azureml.core.model import Model, InferenceConfig
from azureml.core import Environment
from azureml.core.webservice import AciWebservice

subscription_id = '81cefad3-d2c9-4f77-a466-99a7f541c7bb'
resource_group = 'aml-quickstarts-260697'
workspace = 'quick-starts-ws-260697'

#print(remote_run.get_environment())

#env = Environment.from_conda_specification(name='sklearn-1.0:37', file_path='conda_dependencies.yml')
#env = Environment.get(workspace = ws, name= "heart-automl-env")

# get the best run
best_run, best_model = remote_run.get_output()
# get the environment from the best run
env = best_run.get_environment()
inference_config = InferenceConfig(entry_script='score.py', environment=env)

#inf_config = InferenceConfig(entry_script='score.py', environment=env, conda_file="conda_dependencies.yml")
#inf_config = InferenceConfig(entry_script='score.py', environment=env)

#deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4, enable_app_insights=True, auth_enabled=True)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)

service = Model.deploy(workspace = ws, 
                    name = "heart-automl-deploy",
                    models = [model],
                    inference_config= inference_config,
                    deployment_config=deployment_config,
                    overwrite = True)

# ml_client = MLClient(
#     DefaultAzureCredential(), subscription_id, resource_group, workspace
# )

# # Define an endpoint name
# endpoint_name = "heart-endpoint"

# # Example way to define a random name
# import datetime

# endpoint_name = "endpt-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# # create an online endpoint
# endpoint = ManagedOnlineEndpoint(
#     name = endpoint_name, 
#     description="This is a heart data endpoint",
#     auth_mode="key"
# )

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

TODO: In the cell below, send a request to the web service you deployed to test it.

In [18]:
%run endpoint.py

MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
